# DeepMatcher

In [3]:
!pip install deepmatcher 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 1.8/2.0 MB 10.4 kB/s eta 0:00:13^C
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 1.8/2.0 MB 10.4 kB/s eta 0:00:13
ERROR: Operation cancelled by user


## Training

In [ ]:
import pickle
all_claims = pickle.load(open('../../data/corona/corona_allClaims','rb'))
user_claims = pickle.load(open('../../data/corona/user_claims.pkl','rb'))
all_tables = pickle.load(open('../../data/corona/corona_tables','rb'))
ground_truth = pickle.load(open('../../data/corona/corona_GT.pkl','rb'))

In [ ]:
all_rows = []
for claim in all_claims:
  try:
    all_rows.append(' '.join(all_claims[claim]))
  except:
    for r in all_claims[claim]:
      all_rows.append(' '.join(r))

In [ ]:
import numpy as np
import pandas as pd

def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test


In [ ]:
import random
header = ['left_column','right_column','label']
all_table = []

for claim in [m for m in ground_truth.keys()][0:int(0.99*len(ground_truth))]:
#for claim in [m for m in user_claims.keys()][0:int(0.5*len(user_claims))]:

    row = []
    for r in ground_truth[claim]:
        all_table.append([claim,r,'1'])
        
    for r in random.sample(all_tables,len(all_tables)):
        if ' '.join(r) not in ground_truth[claim]:
            all_table.append([claim,' '.join(r),'0'])


In [ ]:
from sklearn.utils import shuffle

df = pd.DataFrame(all_table,columns=header)
df = shuffle(df)
train, test, val = train_validate_test_split(df)

In [ ]:
df

,left_movie,right_movie,label
17481,the number of covid-19 total confirmed cases i...,total recovered slovenia 0 0 10 233 1358 1358,0
19484,there are 1 million cases in china,total deaths iran 0 43 2898 6028 7797 7942,0
27297,greece has 20 deaths,new recovered cabo verde 0 0 0 4 189 44,0
17848,the number of covid-19 total confirmed cases i...,new recovered fiji 0 0 0 12 3 0,0
2825,the total death in russia is below 10000.,new recovered israel 0 1 223 8337 6251 128,0
...,...,...,...
22967,the death rate in us is higher than italy,total deaths mauritania 0 0 1 1 23 31,0
25427,43 new deaths in the netherlands,new confirmed bangladesh 0 0 51 7616 39486 5292,0
15330,the recovery rate in italy is much higher than...,new confirmed monaco 0 1 51 43 4 0,0
8273,there were 30 total deaths in spain on march 12,total confirmed madagascar 0 0 57 128 771 845,0


In [ ]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)
val.to_csv('validate.csv',index=False)

In [ ]:
import deepmatcher as dm
train, validation, test = dm.data.process(
    path='',
    train='train.csv',
    validation='validate.csv',
    test='test.csv')



Reading and processing data from "train.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from "validate.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#######################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [ ]:
model = dm.MatchingModel()
model.run_train(train, validation, best_save_path='deepMatcher_corona.pth',epochs=2)

* Number of trainable parameters: 2798703
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:770: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   22.8 | Load Time:   10.2 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 701.60

===>  EVAL Epoch 1


0% [██████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 1 || Run Time:    1.4 | Load Time:    1.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 1060.43

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 2 || Run Time:   23.3 | Load Time:   10.4 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 686.24

===>  EVAL Epoch 2


0% [██████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 2 || Run Time:    1.5 | Load Time:    1.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 1043.30

---------------------

Loading best model...
Training done.


tensor(0., device='cuda:0')

## Testing

In [ ]:
import random
from sklearn.utils import shuffle
import pandas as pd
from tqdm import tqdm

header = ['left_column','right_column','label']
deep_corona = {}
for claim in tqdm([m for m in ground_truth.keys()][int(0.6*len(ground_truth))::]):
#for claim in tqdm([m for m in user_claims.keys()][int(0.5*len(user_claims))::]):

    all_table = []

    row = []
    for r in ground_truth[claim]:
        all_table.append([claim, r,'1'])
        
    for r in random.sample(all_tables,len(all_tables)):
        if ' '.join(r) not in ground_truth[claim]:
            all_table.append([claim,' '.join(r),'0'])

    dff = pd.DataFrame(all_table,columns=header)          
    dff = shuffle(dff)
    dff["id"] = dff.index

    dff.to_csv('new_test.csv',index=False)
    rev_index = pd.Series(dff.right_movie,index=dff.index).to_dict()    


    unlabeled = dm.data.process_unlabeled(path='new_test.csv', trained_model=model,ignore_columns=('label'))
    preds = model.run_prediction(unlabeled)


    temp = {}
    for row in preds.iterrows():
      temp[(rev_index[int(row[0])])] =  float(row[1])
    deep_corona[claim] = temp = dict(sorted(temp.items(), key=lambda x: x[1],reverse=True))



  0%|          | 0/4235 [00:00<?, ?it/s]
Reading and processing data from "new_test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 2 || Run Time:    0.3 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



0% [############################# ] 100% | ETA: 00:00:00

...

Finished Epoch 2 || Run Time:    0.4 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 2


0% [█████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

100%|██████████| 4235/4235 [1:20:17<00:00,  1.14s/it]

Finished Epoch 2 || Run Time:    0.4 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [ ]:
for KK in [1,5,20,5000]: 
    i = 0
    MAP, MR, hasP = 0,0,0

    for claim in deep_corona:
        if claim not in ground_truth or len(ground_truth[claim])==0: continue
        if claim in user_claims: continue
        
        i+=1
        preds =  [f for (f,j) in   sorted(deep_corona[claim].items(), key=lambda x: x[1],reverse=True)  ][0:KK]
        golds = [f for f in ground_truth[claim]]

        MAP += MAP_K(golds,preds)
        MR += MRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)



#################### 1 ###########################

MRR: 0.17050133204165657 MAP: 0.16929038508113345 HAS POSITIVE: 0.17050133204165657

#################### 5 ###########################

MRR: 0.18126665052070784 MAP: 0.1808387825946563 HAS POSITIVE: 0.2024703317994672

#################### 20 ###########################

MRR: 0.18730014397833875 MAP: 0.18695454792857702 HAS POSITIVE: 0.27173649794139015

#################### 5000 ###########################

MRR: 0.19397275469896552 MAP: 0.1936748275772534 HAS POSITIVE: 1.0


In [ ]:
for KK in [1,5,20,5000]: 
    i = 0
    MAP, MR, hasP = 0,0,0

    for claim in deep_corona:
        if claim not in ground_truth or len(ground_truth[claim])==0: continue
        if claim not in user_claims: continue
        
        i+=1
        preds =  [f for (f,j) in   sorted(deep_corona[claim].items(), key=lambda x: x[1],reverse=True)  ][0:KK]
        golds = [f for f in ground_truth[claim]]

        MAP += MAP_K(golds,preds)
        MR += MRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)



#################### 1 ###########################

MRR: 0.28 MAP: 0.2 HAS POSITIVE: 0.28

#################### 5 ###########################

MRR: 0.28 MAP: 0.2 HAS POSITIVE: 0.28

#################### 20 ###########################

MRR: 0.31811484593837536 MAP: 0.2476386554621849 HAS POSITIVE: 0.6

#################### 5000 ###########################

MRR: 0.3219331324033309 MAP: 0.25154602878682863 HAS POSITIVE: 1.0
